# Firmware Build Setup

This tutorial will introduce you to the 'simpleserial' communications system. It will show you how to perform different operations on data based on input from the ChipWhisperer software. This can be used for building your own system which you wish to 'break'. All the `%%bash` blocks can be run either in Jupyter or in your favourite command line environment (note that Jupyter resets your path between blocks).

Additionally, there is a block below that has `#Parameters` at the top. This is a block used to configure tutorials for different targets and situations. Make sure you change these parameters in this and future tutorials so that they match your hardware setup. Important parameters include:

* `SCOPETYPE` - Capture hardware to use, for example "OPENADC" for CWLite or CWPro.
* `PLATFORM`  - Target being attacked. For example, "CW308_STM32F3" for STM32F3 on CW308 board.
* `CRYPTO_TARGET` - Cryptography library used for encryption functions. For example, "TINYAES128C".

Some common hardware configurations are (`CRYPTO_TARGET` is unused in this tutorial, but will be used in later tutorials involving AES and RSA):

CWLite 1-part w/ Arm target:
```Python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
```

CWLite 1-part w/ Xmega target
```Python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'
```

CWNano 1-part
```Python
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'TINYAES128C'
```

In [4]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

## What is SimpleSerial

SimpleSerial is the communications protocol used for almost all of the ChipWhisperer demo project. It's a very basic serial protocol which can be easily implemented on most systems. This system communicates using a standard asyncronous serial protocol, 38400 baud, 8-N-1.

All messages are sent in ASCII-text, and are normally terminated with a line-feed ('\n'). This allows you to interact with the simpleserial system over a standard terminal emulator.

`x`
>Sending a 'x' resets the buffers. This does not require a line-feed termination. It is suggested to always send a stream of x's to initilize the system in case the device was already in some other mode due to noise/corruption.

`k00112233445566778899AABBCCDDEEFF\n`
>Loads the encryption key `00112233445566778899AABBCCDDEEFF` into the system. If not called the system may use some default key.

`pAABBCCDDEEFF00112233445566778899\n`
>Encrypts the data `AABBCCDDEEFF00112233445566778899` with the key loaded with the 'k' command. The system will respond with a string starting with r, as shown next.

`rCBBD4A2B34F2571758FF6A797E09859D\n`
>This is the response from the system. If data has been encrypted with a 'p' for example, the system will respond with the 'r' sequence automatically. So sending the earlier example means the result of the encryption was `cbbd4a2b34f2571758ff6a797e09859d`.

## Building the Basic Example

To bulid the basic example, you'll need an appropriate compiler for your target. For the ChipWhisperer Lite/Xmega platform, you'll need `avr-gcc` and `avr-libc`, while if you're using an ARM target (like the ChipWhisperer Lite/STM32 platform), your need the GNU Toolchain for ARM devices. If you're using a target with a different architecture, you'll need to install the relevant compiler. If you're unsure, you can run the block below. If you've got the right stuff installed, you should see some version and copyright info printed for the relevant compiler:

In [5]:
%%bash
#check for avr-gcc
avr-gcc --version

#check for ARM gcc
arm-none-eabi-gcc --version

avr-gcc.exe (WinAVR 20100110) 4.3.3
Copyright (C) 2008 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

arm-none-eabi-gcc.exe (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Now that you have the relevant toolchain installed, you should be able to build firmware for your desired platform. We'll begin by creating a new project based on simpleserial-base by making a new firmware and copying the files from the project we want to work on:

In [6]:
%%bash
cd ../hardware/victims/firmware/
mkdir -p simpleserial-base-lab1 && cp -r simpleserial-base/* $_
cd simpleserial-base-lab1

Next we'll build the firmware. You'll need to specify the `PLATFORM` and `CRYPTO_TARGET` for your target. To save you from having to re-enter this info in every make block, you can edit the python below with your platform and crypto_target.

Common platforms are CWLITEXMEGA and CWLITEARM. To see a list of platforms leave `PLATFORM` as is.

This tutorial doesn't use any crypto, so we can leave `CRYPTO_TARGET` as `NONE`.

In [7]:
CRYPTO_TARGET = "NONE"

Provided you completed the fields above, you should be able to successfully run the block below.

In [5]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-base-CWLITEARM.hex
rm -f -- simpleserial-base-CWLITEARM.eep
rm -f -- simpleserial-base-CWLITEARM.cof
rm -f -- simpleserial-base-CWLITEARM.elf
rm -f -- simpleserial-base-CWLITEARM.map
rm -f -- simpleserial-base-CWLITEARM.sym
rm -f -- simpleserial-base-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-base.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-base.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compili

## Modifying the Basic Example

At this point we want to modify the system to perform 'something' with the data, such that we can confirm the system is working. To do so, open the file `simpleserial-base.c` in the simpleserial-base-lab1 folder with a code editor such as Programmer's Notepad (which ships with WinAVR).

Find the following code block towards the end of the file:

```C
/**********************************
 * Start user-specific code here. */
trigger_high();

//16 hex bytes held in 'pt' were sent
//from the computer. Store your response
//back into 'pt', which will send 16 bytes
//back to computer. Can ignore of course if
//not needed

trigger_low();
/* End user-specific code here. *
```

Now modify it to increment the value of each data byte:

```C
/**********************************
 * Start user-specific code here. */
trigger_high();

//16 hex bytes held in 'pt' were sent
//from the computer. Store your response
//back into 'pt', which will send 16 bytes
//back to computer. Can ignore of course if
//not needed

for(int i = 0; i < 16; i++){
    pt[i]++;
}

trigger_low();
/* End user-specific code here. *
 ********************************/
```

Then rebuild the file with `make`:

In [6]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-base-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-base-CWLITEARM.hex
rm -f -- simpleserial-base-CWLITEARM.eep
rm -f -- simpleserial-base-CWLITEARM.cof
rm -f -- simpleserial-base-CWLITEARM.elf
rm -f -- simpleserial-base-CWLITEARM.map
rm -f -- simpleserial-base-CWLITEARM.sym
rm -f -- simpleserial-base-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-base.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-base.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compili

## Python Script

We'll end by uploading the firmware onto the target and communicating with it via a python script. Depending on your target, uploading firmware will be different. For the XMega and STM32 targets, you can use ChipWhisperer's interface. Otherwise, you'll likely need to use and external programmer. If you have a CW1173/Xmega board, you can run the following blocks without modification. After running the final block, you should see two sets of hexadecimal numbers, with the second having values one higher than the first.

We'll begin by importing the ChipWhisperer module. This will allow us to connect to and communicate with the ChipWhisperer hardware. The ChipWhisperer module also includes analysis software, which we'll be looking at in later tutorials.

In [1]:
import chipwhisperer as cw

Documentation is available by calling `help()` on the module, submodules, functions, etc.:

In [14]:
help(cw)
help(cw.scopes.OpenADC)

Help on package chipwhisperer:

NAME
    chipwhisperer

DESCRIPTION
    .. module:: chipwhisperer
       :platform: Unix, Windows
       :synopsis: Test
    
    .. moduleauthor:: NewAE

PACKAGE CONTENTS
    analyzer (package)
    capture (package)
    common (package)
    hardware (package)
    tests (package)

SUBMODULES
    CWFirmwareUpdate
    aes128leakage
    cpa_algorithms
    key_text_patterns
    ktp
    preprocessing
    programmers
    scopes
    targets

FUNCTIONS
    analyzerPlots(attack_results=None)
        Create an object to get plot data for analyzer results
    
    cpa(trace_source, leak_model=None, algorithm=<class 'chipwhisperer.analyzer.attacks.cpa_algorithms.progressive.CPAProgressive'>)
        Create a CPA attack object with sane defaults
        
        Create a CPA attack object with:
        
        * trace source = trace_source
        * traces start at beginning
        * uses all traces for attack
        * 1 iteration
        * Reporting Interval = 10

Next we'll need to connect to the scope end of the hardware. There's a few different types of scopes we can connect to which can all be specified via `cw.scopes.<scope_type>`. `cw.scopes.OpenADC` is the default and is used for the CWLite and CWPro. If you're using another scope (such as the Nano), you'll need to pass a different scope type to `cw.scope()` (`cw.scopes.CWNano` in the CWNano case).

In [12]:
scope = None
if SCOPETYPE == "OPENADC":
    scope = cw.scope(cw.scopes.OpenADC)
elif SCOPETYPE == "CWNANO":
    scope = cw.scope(cw.scopes.CWNano)

In [15]:
help(scope.adc)

Help on TriggerSettings in module chipwhisperer.capture.scopes._OpenADCInterface object:

class TriggerSettings(chipwhisperer.common.utils.parameter.Parameterized, chipwhisperer.common.utils.util.DisableNewAttr)
 |  TriggerSettings(oaiface)
 |  
 |  Abstract class that implements basic functionality required by parameterized objects.
 |  All parameterized objects should have _name overriden in the class. The objects can also override this attribute
 |   with different names in the case of having two or more instances. The _description is optional.
 |  
 |  Method resolution order:
 |      TriggerSettings
 |      chipwhisperer.common.utils.parameter.Parameterized
 |      chipwhisperer.common.utils.util.DisableNewAttr
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, oaiface)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  extTri

We'll also need to setup the interface to the target (typically what we want to attack). Like with scopes, there's a few different interfaces we can use, which are available through `scope.targets.<target_type>`. The default, SimpleSerial, communicates over UART and is almost always the correct choice.

In [9]:
target = cw.target(scope, cw.targets.SimpleSerial)

Next, we'll do some basic setup. Most of these settings don't matter for now, but take note of the `scope.clock` and `scope.io`, which setup the clock and serial io lines, which needs to be done before programming the target. 

**Some targets require settings different than what's below. Check the relevant wiki article for your target for more information**

In [24]:
# setup scope parameters
if SCOPETYPE == "OPENADC":
    scope.gain.db = 45
    scope.adc.samples = 3000
    scope.adc.offset = 1250
    scope.adc.basic_mode = "rising_edge"
    scope.clock.clkgen_freq = 7370000
    scope.clock.adc_src = "clkgen_x4"
    scope.trigger.triggers = "tio4"
    scope.io.tio1 = "serial_rx"
    scope.io.tio2 = "serial_tx"
    scope.io.hs2 = "clkgen"
elif SCOPETYPE == "CWNANO":
    scope.io.clkout = 7370000
    scope.adc.clk_freq = 7370000
    scope.io.tio1 = "serial_rx"
    scope.io.tio2 = "serial_tx"

In [25]:
print(scope.gain)

mode = high
gain = 77
db   = 55.12890625



Now that the clock and IO lines are setup, we can program the target. ChipWhisperer includes a generic programming function, `cw.programTarget(scope, type, fw_path)`. Here `type` is one of the programmers available in the cw.programmers submodule (`help(cw.programmers)` for more information). `fw_path` is the path to the hex file that you want to flash onto the device.

The final part of the binary path should match your platform (`<path>/simpleserial-base-CWLITEARM.hex` for CWLITEARM)

In [12]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None
    
fw_path = '../hardware/victims/firmware/simpleserial-base-lab1/simpleserial-base-{}.hex'.format(PLATFORM)

And finally actually programming the device:

In [13]:
cw.programTarget(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 4671 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4671 bytes


Finally, we'll load some text, send it to the target, and read it back. We also capture a trace here, but don't do anything with it yet (that will come in later tutorials). You should see your original text with the received text below it.

In [14]:
ktp = cw.ktp.Basic(target=target) # object to generate fixed/random key and text (default fixed key, random text)
key, text = ktp.newPair()  # get our fixed key and random text

target.loadInput(text)
scope.arm()
target.go() #send key/text to target
timeout = 50
# wait for target to finish
while target.isDone() is False and timeout:
    timeout -= 1
    time.sleep(0.01)
ret = scope.capture()
num_char = target.ser.inWaiting()
output = target.ser.read(num_char, timeout=10)

import binascii
print(binascii.b2a_hex(text).upper())
print(output[1:-5].encode("ascii"))

b'2BEAB346BB795AB378D0F81793E0D678'
b'2BEAB346BB795AB378D0F81793E0D678'


In [11]:
print(scope.getLastTrace())

[]


Now that we're done with this tutorial, we'll need to disconnect from the ChipWhisperer. This will prevent this session from interferening from later ones (most notably with a `USB can't claim interface` error). Don't worry if you forget, unplugging and replugging the ChipWhipserer should fix it.

In [ ]:
scope.dis()
target.dis()

## Future Tutorials

The next tutorials that you run will start using helper scripts to make setup a little faster and more consistent between tutorials. Those scripts run mostly the same setup code that we did here, but if you'd like to see exactly what they're doing, they're all included in the `Helper_Scripts` folder.

For example, the scope setup (gain, clock, etc) is taken care of by `Helper Scripts/Setup_Target_Generic.ipynb`.